### Objective of this assignment is to extract textual data from SEC / EDGAR financial reports and perform text analysis to compute different variables 

1.	All input variables in “cik_list.xlsx”
2.	positive_score
3.	negative_score
4.	polarity_score
5.	average_sentence_length
6.	percentage_of_complex_words
7.	fog_index
8.	complex_word_count
9.	word_count
10.	uncertainty_score
11.	constraining_score
12.	positive_word_proportion
13.	negative_word_proportion
14.	uncertainty_word_proportion
15.	constraining_word_proportion
16.	constraining_words_whole_report


## Importing different libraries used

In [2]:
import numpy as np
import pandas as pd
import nltk
import random
import string
import bs4 as bs
import urllib.request
from urllib.request import Request, urlopen
import re
import requests

## Importing all files from the drive used in the code 

In [3]:
#import cik_list excel file for inputs
input_file_df = pd.read_excel("/content/drive/MyDrive/Doc/Blackcoffer_task/cik_list.xlsx", dtype=object)

#import another excel file for output file format
output_file_df = pd.read_excel("/content/drive/MyDrive/Doc/Blackcoffer_task/Output Data Structure.xlsx")


#import sentimentalword file (positive and negative files)
positive_df = pd.read_excel("/content/drive/MyDrive/Doc/Blackcoffer_task/SentimentWordLists.xlsx", sheet_name="Positive", header=None, dtype=object)
negative_df = pd.read_excel("/content/drive/MyDrive/Doc/Blackcoffer_task/SentimentWordLists.xlsx", sheet_name="Negative", header=None, dtype=object)

#uncertainity dictionary file
uncertainity_df = pd.read_excel("/content/drive/MyDrive/Doc/Blackcoffer_task/uncertainty_dictionary.xlsx", header=None)

#constraining dictionary file
constraining_df = pd.read_excel("/content/drive/MyDrive/Doc/Blackcoffer_task/constraining_dictionary.xlsx", header=None)

In [4]:
input_file_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   CIK       152 non-null    object        
 1   CONAME    152 non-null    object        
 2   FYRMO     152 non-null    object        
 3   FDATE     152 non-null    datetime64[ns]
 4   FORM      152 non-null    object        
 5   SECFNAME  152 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 7.2+ KB


In [5]:
input_file_df['SECFNAME']

0       edgar/data/3662/0000950170-98-000413.txt
1       edgar/data/3662/0000950170-98-001001.txt
2       edgar/data/3662/0000950172-98-000783.txt
3       edgar/data/3662/0000950170-98-002145.txt
4       edgar/data/3662/0000950172-98-001203.txt
                         ...                    
147    edgar/data/12239/0001104659-07-024804.txt
148    edgar/data/12239/0001104659-07-040463.txt
149    edgar/data/12239/0001104659-07-041441.txt
150    edgar/data/12239/0001104659-07-042333.txt
151    edgar/data/12239/0001104659-07-062470.txt
Name: SECFNAME, Length: 152, dtype: object

### Add https://www.sec.gov/Archives/ to every cells of column SECFNAME of (cik_list.xlsx) as prefix

In [6]:
# string to add as prefix
prefix_toadd = "https://www.sec.gov/Archives/";

#concate of two string urls
input_file_df['SECFNAME'] = prefix_toadd + input_file_df['SECFNAME'];

In [7]:
#check the link again
input_file_df['SECFNAME'][0]

'https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-000413.txt'

In [8]:
#check for null values in the input file
input_file_df.isnull().values.any()

False

In [9]:
# check no of columns in the output file
output_file_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   CIK                              0 non-null      object
 1   CONAME                           0 non-null      object
 2   FYRMO                            0 non-null      object
 3   FDATE                            0 non-null      object
 4   FORM                             0 non-null      object
 5   SECFNAME                         0 non-null      object
 6   positive_score                   0 non-null      object
 7   negative_score                   0 non-null      object
 8   polarity_score                   0 non-null      object
 9   average_sentence_length          0 non-null      object
 10  percentage_of_complex_words      0 non-null      object
 11  fog_index                        0 non-null      object
 12  complex_word_count               0 non-null      obje

In [10]:
# drop the columns matches to input file from out file for smooth joining of two tables
output_df = output_file_df.drop(['CIK','CONAME','FYRMO','FDATE','FORM','SECFNAME'],axis = 'columns')

In [11]:
# join two tables to create a final table
final_file_df = input_file_df.join(output_df)

In [12]:
# final file columns name
final_file_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   CIK                              152 non-null    object        
 1   CONAME                           152 non-null    object        
 2   FYRMO                            152 non-null    object        
 3   FDATE                            152 non-null    datetime64[ns]
 4   FORM                             152 non-null    object        
 5   SECFNAME                         152 non-null    object        
 6   positive_score                   0 non-null      object        
 7   negative_score                   0 non-null      object        
 8   polarity_score                   0 non-null      object        
 9   average_sentence_length          0 non-null      object        
 10  percentage_of_complex_words      0 non-null      object       

## Dwonload ** Punkt ** Sentence Tokenizer. This divides a text into a list of sentences

In [13]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Defining a bot function for the all operations and a loop for performing those operations to all rows 

In [14]:
i = 0
def botfn(i):
  try:
    while (i < len(final_file_df['SECFNAME'])):
      raw_html = urllib.request.urlopen(final_file_df['SECFNAME'][i])
      raw_html = raw_html.read()
      article_html = bs.BeautifulSoup(raw_html,'lxml')
      article_paragraphs = article_html.find_all('p')
      article_text = ''

      for para in article_paragraphs:
        article_text +=para.text

      corpus = nltk.sent_tokenize(article_text)
      for j in range (len(corpus)):
        corpus[j] = corpus[j].lower()
        corpus[j] = re.sub(r'\W',' ',corpus[j])
        corpus[j] = re.sub(r'\s+',' ',corpus[j])
        corpus[j] = re.sub(r'[0-9]', '', corpus[j])
      # for total sentences use len(corpus)

      from nltk.tokenize import word_tokenize

      #Cleaning sentences by removing stopwords 
      # filtering sentences 
      corpus_filtered = []
      for j in range(len(corpus)):
        tokens = word_tokenize(corpus[j])
        corpus_filtered.extend(tokens)

      with open("/content/drive/MyDrive/Doc/Blackcoffer_task/StopWords_GenericLong.txt", "r") as stopwordsfile:
        stopwords = stopwordsfile.read().splitlines()
      filtered_corpus_stpwrds = [word for word in corpus_filtered if not word in stopwords]

      # Calculating Positive and Negative wordcore
      # poitive words and entry in final_file_df table
      positive_list = positive_df[0].tolist()
      for j in range(len(positive_df)):
        positive_list[j] = positive_list[j].lower()
      positive_words = [word for word in filtered_corpus_stpwrds if word in positive_list]
      positive_score = len(positive_words)
      final_file_df['positive_score'][i] = positive_score

      # negative words and entry in final_file_df table
      negative_list = negative_df[0].tolist()
      for j in range(len(negative_df)):
        negative_list[j] = negative_list[j].lower()
      negative_words = [word for word in filtered_corpus_stpwrds if word in negative_list]
      negative_score = len(negative_words)
      final_file_df['negative_score'][i] = negative_score

      # Calculation of polarity score and entry in final_file_df table
      Polarity_Score = (positive_score - negative_score)/((positive_score + negative_score) + 0.000001)

      
      if ( Polarity_Score < -0.5):                            # defining class on the basis of polarity score range
        final_file_df['polarity_score'][i] = "Most Negative"
      elif ( Polarity_Score >= -0.5 and Polarity_Score < 0):
        final_file_df['polarity_score'][i] = "Negative"
      elif ( Polarity_Score == 0):
        final_file_df['polarity_score'][i] = "Neutral"
      elif ( Polarity_Score > 0 and Polarity_Score < 0.5):
        final_file_df['polarity_score'][i] = "Positive"
      elif ( Polarity_Score >= 0.5 ):
        final_file_df['polarity_score'][i] = "Most Positive"

      # Calculating average_sentence_length and entry in final_file_df table
      average_sentence_length = len(corpus_filtered)/len(corpus) # Exception - division by zero
      final_file_df['average_sentence_length'][i] = average_sentence_length

      # Calculating complex words and entry in final_file_df table
      vowels = "aeiou"
      complex_word_count = 0
      for word in filtered_corpus_stpwrds:
        if (word.endswith(("es","ed","el"))):
          counter = 0
        else:
          counter = 0
          for c in word:
            if (c in vowels):
              counter += 1
        if (counter > 2):
          complex_word_count += 1
      
      final_file_df['complex_word_count'][i] = complex_word_count
      final_file_df['percentage_of_complex_words'][i] = complex_word_count / len(filtered_corpus_stpwrds)

      # Calculating fog_index and entry in final_file_df table
      
      fog_index = 0.4 * (average_sentence_length + (complex_word_count/len(filtered_corpus_stpwrds))) # formula used : 0.4 * (Average Sentence Length + Percentage of Complex words)
      final_file_df['fog_index'][i] = fog_index

      # Calculating wordcount and entry in final_file_df table
      wordcount = len(corpus_filtered)
      final_file_df['word_count'][i] = wordcount
      
      # Calculating uncertainty_score and entry in final_file_df table
      uncertainity_list = uncertainity_df[0].tolist()
      for j in range(len(uncertainity_list)):
        uncertainity_list[j] = uncertainity_list[j].lower()
      uncertainity_words = [word for word in filtered_corpus_stpwrds if word in uncertainity_list]
      uncertainity_score = len(uncertainity_words)
      final_file_df['uncertainty_score'][i] = uncertainity_score

      # Calculating constraining_score and entry in final_file_df table
      constraining_list = constraining_df[0].tolist()
      for j in range(len(constraining_list)):
        constraining_list[j] = constraining_list[j].lower()
      constraining_words = [word for word in filtered_corpus_stpwrds if word in constraining_list]
      constraining_score = len(constraining_words)
      final_file_df['constraining_score'][i] = constraining_score

      # Calculating positive_word_proportion and entry in final_file_df table
      positive_word_proportion = positive_score/wordcount
      final_file_df['positive_word_proportion'][i] = positive_word_proportion
      
      # Calculating negative_word_proportion and entry in final_file_df table
      negative_word_proportion = negative_score/wordcount
      final_file_df['negative_word_proportion'][i] = negative_word_proportion
      
      # Calculating uncertainty_word_proportion and entry in final_file_df table
      uncertainty_word_proportion = uncertainity_score/wordcount
      final_file_df['uncertainty_word_proportion'][i] = uncertainty_word_proportion
      
      # Calculating constraining_word_proportion and entry in final_file_df table
      constraining_word_proportion = constraining_score/wordcount
      final_file_df['constraining_word_proportion'][i] = constraining_word_proportion
      
      # Calculating constraining_words_whole_report and entry in final_file_df table
      constraining_words_whole = [word for word in corpus_filtered if word in constraining_list]
      constraining_score_whole_report = len(constraining_words_whole)
      final_file_df['constraining_words_whole_report'][i] = constraining_score_whole_report      
      
      i = i + 1

  except urllib.error.HTTPError as exception:
    botfn(i)

  except ZeroDivisionError as exception:
    final_file_df['positive_score'][i]= 0
    final_file_df['negative_score'][i]= 0
    final_file_df['polarity_score'][i]= 0
    final_file_df['average_sentence_length'][i]= 0
    final_file_df['word_count'][i]= 0
    final_file_df['uncertainty_score'][i]= 0
    final_file_df['constraining_score'][i]= 0
    final_file_df['positive_word_proportion'][i]= 0
    final_file_df['positive_word_proportion'][i]= 0
    final_file_df['negative_word_proportion'][i]= 0
    final_file_df['uncertainty_word_proportion'][i]= 0
    final_file_df['constraining_word_proportion'][i]= 0
    final_file_df['constraining_word_proportion'][i]= 0
    final_file_df['complex_word_count']=0
    final_file_df['percentage_of_complex_words']= 0
    final_file_df['fog_index']= 0
    i += 1
    botfn(i)
botfn(i)
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to b

## Creating a csv file of the final_file_df and save to drive


In [15]:
with open('/content/drive/MyDrive/Doc/Blackcoffer_task/Blackcoffer_Submission_File.csv', 'w') as f:
  final_file_df.to_csv(f)